In [ ]:
# useing this variable for saving models and summaries
exp_name = "RCNN"

## Running CNN then RNN
5 layers CNN [2, 2, 2, 2, 2], then RNN, bias_shape = [32,64,64, 64, 64]

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../src')
sys.path.append('../src/models')

In [ ]:
import models.CNN_lstm_model as CNN_lstm_model
import training  
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [ ]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

## Model

In [ ]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 10

pooling_stride = [2, 2, 2, 2, 2]
weights_shape = [[5,n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]

bias_shape = [32,64,64, 64, 64]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 1


#positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [ ]:
pos_weight

In [ ]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 


In [ ]:
model = CNN_lstm_model.Model(n_variables, n_classes)

In [ ]:
model.build_cnn_lstm_net(pooling_stride,
                         weights_shape,
                         bias_shape,
                         activation,
                         lstm_n_hidden,
                         n_classes,
                         learning_rate,
                         pos_weight)

In [ ]:
train_obj = training.Trainer(model,
                 train_data, train_gt,
                 validation_data, validation_gt,
                 sess,
                 summary_path,
                 batch_size,
                 global_step = 0)

In [ ]:
train_obj.train(max_iter = 10000,
                train_eval_step=100,
                validation_eval_step=100,
                display_step=100)

In [ ]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=train_obj.global_step)

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-1000')

In [ ]:
probability_of_pos = nn_eval.evaluate_model(model,
                                   sess,
                                   test_data,
                                   test_gt)

In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})